<a href="https://colab.research.google.com/github/muneeebelahi/Forest-Damage-Detection-using-unsupervised-learnring/blob/main/ForestDamageDetection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Clustering on Landsat 8 data.

In [2]:
import ee
ee.Authenticate()
ee.Initialize()
#Authentication key for this week 4/1AWtgzh6cKVTwrvrvLuCLv21H7C90zN4c-PNViS4MWdXJisI11Jh_kupViag

To authorize access needed by Earth Engine, open the following URL in a web browser and follow the instructions. If the web browser does not start automatically, please manually browse the URL below.

    https://code.earthengine.google.com/client-auth?scopes=https%3A//www.googleapis.com/auth/earthengine%20https%3A//www.googleapis.com/auth/devstorage.full_control&request_id=QLvew1U6PiEcI3B2IBymHSkSgNXKaHURknDP5BkOjKo&tc=er1wscpIjhpOgCF_nReoNZyJR0VIOWkkjyaC-WTxSf8&cc=OsAQCWaepyE94Fnn8loEMkoxOe0CDW7bFlYHdVBAZYw

The authorization workflow will generate a code, which you should paste in the box below.
Enter verification code: 4/1AWtgzh6cKVTwrvrvLuCLv21H7C90zN4c-PNViS4MWdXJisI11Jh_kupViag

Successfully saved authorization token.


In [14]:
import numpy as np
from sklearn.cluster import KMeans
import matplotlib.pyplot as plt
import seaborn as sns
# Load the required packages for unsupervised machine learning
from sklearn.cluster import KMeans
import numpy as np
import folium


ModuleNotFoundError: ignored

In [15]:
pip install geehydro

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Preparing metadata (setup.py) ... done
  Created wheel for geehydro: filename=geehydro-0.2.0-py2.py3-none-any.whl size=10140 sha256=ddf3e5689785cc929e4a07b5dcfd545157d434eca22fde079f75c919ea8d066f
  Stored in directory: /root/.cache/pip/wheels/cc/f0/c8/f761318de15d20d46693d576710a46d56b7bd9ba15249800e7
Successfully built geehydro


In [16]:
import geehydro

In [4]:
#defining a region of interest
roi = ee.Geometry.Polygon(
    [[[73.393569, 33.659383],
      [73.384694, 33.661351],
      [73.383522, 33.660623],
      [73.380984, 33.660574],
      [73.376924, 33.659139],
      [73.375569, 33.658225],
      [73.375275, 33.659185],
      [73.374458, 33.658688],
      [73.369600, 33.658287],
      [73.368559, 33.653864],
      [73.368546, 33.652275],
      [73.368042, 33.650022],
      [73.367986, 33.648832],
      [73.370205, 33.649609],
      [73.370601, 33.650278],
      [73.373957, 33.649403],
      [73.375298, 33.649529],
      [73.376547, 33.645811],
      [73.379243, 33.645544],
      [73.379898, 33.645298],
      [73.380495, 33.644003],
      [73.381702, 33.642400],
      [73.382661, 33.642430],
      [73.382732, 33.643780],
      [73.384380, 33.647222],
      [73.386612, 33.648572],
      [73.390418, 33.649457],
      [73.392329, 33.650895],
      [73.392878, 33.651514],
      [73.393848, 33.651837],
      [73.394421, 33.652418],
      [73.391725, 33.654769],
      [73.393594, 33.659391]]]);

#Zoom in to the region of interest
#center = ee.Geometry.Point([73.381615, 33.653263])


In [25]:
roi = ee.Geometry.Polygon([[[-122.45, 37.75], [-122.45, 37.80], [-122.40, 37.80], [-122.40, 37.75], [-122.45, 37.75]]]);


In [11]:
# Load the data into an Earth Engine Image Collection
img_coll = ee.ImageCollection('LANDSAT/LC08/C01/T1_TOA')

# Filter the image collection based on the defined ROI
filtered_col = img_coll.filterBounds(roi)

# Get the list of band names in the filtered image collection
band_names = filtered_col.first().bandNames().getInfo()

# Check if the required bands are present in the filtered image collection
if "B4" in band_names and "B5" in band_names:
  # Select the red and near-infrared bands
  red_band = filtered_col.select("B4")
  nir_band = filtered_col.select("B5")

  def calculate_ndvi(img):
    # Select the red and near-infrared bands
    red_band = img.select("B4")
    nir_band = img.select("B5")

    # Calculate NDVI
    ndvi = (nir_band.subtract(red_band)).divide(nir_band.add(red_band))

    # Add the NDVI band to the image
    return img.addBands(ndvi.rename("NDVI"))

  # Apply the calculate_ndvi function to each image in the filtered_col ImageCollection
  calculated_col = filtered_col.map(calculate_ndvi)

  # Get the NDVI band from the calculated collection
  ndvi_band = calculated_col.select("NDVI")

  # Calculate the mean value of the NDVI band in the defined ROI
  mean_img = ndvi_band.mean()

  # Convert the mean image to a numpy array
  mean_array = mean_img.reduceRegion(reducer=ee.Reducer.toList(), geometry=roi, scale=30)

  # Extract the mean NDVI value from the numpy array
  mean_value = mean_array.get("NDVI")
  mean_value = mean_value.getInfo()

  # Create a numpy array from the mean NDVI value
  ndvi_array = np.array([mean_value])

  # Reshape the numpy array to be two-dimensional
  ndvi_array = ndvi_array.reshape(-1, 1)

  # Perform unsupervised machine learning using the KMeans algorithm
  kmeans = KMeans(n_clusters=3)
  kmeans.fit(ndvi_array)

  # Obtain the cluster labels for each observation in the numpy array
  labels = kmeans.labels_

  # Print the cluster labels
  print(len(labels))
  print(labels)
else:
  print("B4 and B5 bands not present in the image collection.")


3995
[1 1 1 ... 2 2 2]


In [26]:
lat=73.381615
lon=33.653263
zoom=10
map = folium.Map(location=[lat, lon], zoom_start=zoom)
# Load the data into an Earth Engine Image Collection
img_coll = ee.ImageCollection('LANDSAT/LC08/C01/T1_TOA')

# Filter the image collection based on the defined ROI
filtered_col = img_coll.filterBounds(roi)

# Get the list of band names in the filtered image collection
band_names = filtered_col.first().bandNames().getInfo()

# Check if the required bands are present in the filtered image collection
if "B4" in band_names and "B5" in band_names:
  # Select the red and near-infrared bands
  red_band = filtered_col.select("B4")
  nir_band = filtered_col.select("B5")

  def calculate_ndvi(img):
    # Select the red and near-infrared bands
    red_band = img.select("B4")
    nir_band = img.select("B5")

    # Calculate NDVI
    ndvi = (nir_band.subtract(red_band)).divide(nir_band.add(red_band))

    # Add the NDVI band to the image
    return img.addBands(ndvi.rename("NDVI"))

  # Apply the calculate_ndvi function to each image in the filtered_col ImageCollection
  calculated_col = filtered_col.map(calculate_ndvi)

  # Select the band of interest from the image collection
  band_select = calculated_col.select("NDVI")

  # Calculate the mean value of the band in the defined ROI
  mean_img = band_select.mean()

  # Convert the mean image to a numpy array
  mean_array = mean_img.reduceRegion(reducer=ee.Reducer.toList(), geometry=roi, scale=30)

  # Extract the mean NDVI value from the numpy array
  mean_value = mean_array.get("NDVI")
  mean_value = mean_value.getInfo()

  # Create a numpy array from the mean NDVI value
  ndvi_array = np.array([mean_value])

  # Reshape the numpy array to be two-dimensional
  ndvi_array = ndvi_array.reshape(-1, 1)

  # Perform unsupervised machine learning using the KMeans algorithm
  kmeans = KMeans(n_clusters=3)
  kmeans.fit(ndvi_array)

  # Obtain the cluster labels for each observation in the numpy array
  labels = kmeans.labels_

  # Calculate the mean NDVI value for each cluster
  cluster_means = []
  for i in range(3):
    mask = (labels == i)
    cluster_mean = np.mean(ndvi_array[mask])
    cluster_means.append(cluster_mean)
    print("Cluster {} mean NDVI: {}".format(i, cluster_mean))

  # Visualize the mean NDVI value for each cluster on the map
  # Create a layer for each cluster and set its color based on the mean NDVI value
  #layer = ee.Image(0).expression("if(b == " + str(i) + ", " + str(cluster_mean) + ", 0)").where(0, cluster_mean)
  # Convert the labels from a numpy array to a Python list
  labels_list = labels.tolist()

  # Create an ee.Array from the labels list
  labels_array = ee.Array(labels_list)

  # Reshape the ee.Array to be two-dimensional
  labels_array = labels_array.reshape([ndvi_array.shape[0], 1])
  labels_int = labels.astype(int)
  labels_image = ee.Image.constant(labels_int)
  # Create an ee.Image from the ee.Array
  labels_image = ee.Image.constant(labels_array)
  #layer = ee.Image.constant(cluster_mean).where(labels.eq(i), cluster_mean)
  layer = ee.Image.constant(cluster_mean).where(labels_image.eq(i), cluster_mean)
  

  #map.addLayer(layer, {'min': 0, 'max': 1, 'palette': 'red, green, blue'}, "Cluster " + str(i))
  map.add_child(folium.raster_layers.TileLayer(
    tiles=layer.getMapId(),
    min=0,
    max=1,
    opacity=0.8,
    name="Cluster " + str(i)
).add_to(map))

# Display the map
map.setCenter(lon, lat, zoom)
map.setOptions('HYBRID')
display(map)


Cluster 0 mean NDVI: 0.054158310608408505
Cluster 1 mean NDVI: 0.1473239199833104
Cluster 2 mean NDVI: 0.30146895188850376


EEException: ignored

In [30]:
lat = 33.653263
lon = 73.381615
zoom = 10
map = folium.Map(location=[lat, lon], zoom_start=zoom)
folium.GeoJson(roi.getInfo()).add_to(map)
# Load the data into an Earth Engine Image Collection
img_coll = ee.ImageCollection('LANDSAT/LC08/C01/T1_TOA')

# Filter the image collection based on the defined ROI
filtered_col = img_coll.filterBounds(roi)

# Get the list of band names in the filtered image collection
band_names = filtered_col.first().bandNames().getInfo()

# Check if the required bands are present in the filtered image collection
if "B4" in band_names and "B5" in band_names:
  # Select the red and near-infrared bands
  red_band = filtered_col.select("B4")
  nir_band = filtered_col.select("B5")

  def calculate_ndvi(img):
    # Select the red and near-infrared bands
    red_band = img.select("B4")
    nir_band = img.select("B5")

    # Calculate NDVI
    ndvi = (nir_band.subtract(red_band)).divide(nir_band.add(red_band))

    # Add the NDVI band to the image
    return img.addBands(ndvi.rename("NDVI"))

  # Apply the calculate_ndvi function to each image in the filtered_col ImageCollection
  calculated_col = filtered_col.map(calculate_ndvi)

  # Select the band of interest from the image collection
  band_select = calculated_col.select("NDVI")

  # Calculate the mean value of the band in the defined ROI
  mean_img = band_select.mean()

  # Convert the mean image to a numpy array
  mean_array = mean_img.reduceRegion(reducer=ee.Reducer.toList(), geometry=roi, scale=30)

  # Extract the mean NDVI value from the numpy array
  mean_value = mean_array.get("NDVI")
  mean_value = mean_value.getInfo()

  # Create a numpy array from the mean NDVI value
  ndvi_array = np.array([mean_value])

  # Reshape the numpy array to be two-dimensional
  ndvi_array = ndvi_array.reshape(-1, 1)

  # Perform unsupervised machine learning using the KMeans algorithm
  kmeans = KMeans(n_clusters=3)
  kmeans.fit(ndvi_array)

  # Obtain the cluster labels for each observation in the numpy array
  labels = kmeans.labels_

  # Calculate the mean NDVI value for each cluster
  cluster_means = []
  for i in range(3):
    mask = (labels == i)
    cluster_mean = np.mean(ndvi_array[mask])
    cluster_means.append(cluster_mean)
    print("Cluster {} mean NDVI: {}".format(i, cluster_mean))

  # Visualize the mean NDVI value for each cluster on the map
  # Create a layer for each cluster and set its color based on the mean NDVI value

  # Define the style for each cluster layer
  colors = ['red', 'blue', 'green']

  for label, color in zip(np.unique(labels), colors):
    cluster_mask = labels == label
    cluster_ndvi = ndvi_array[cluster_mask]
    mean_ndvi = cluster_ndvi.mean()

    # Create a layer for the current cluster
    img = ee.Image(mean_ndvi).visualize(min=-1, max=1, palette=[color], opacity=0.7)

    # Add the layer to the map as a TileLayer
    tiles = "https://earthengine.googleapis.com/map/{mapid}/{{z}}/{{x}}/{{y}}?token={token}"
    map_id = img.getMapId()
    folium_layer = folium.TileLayer(
        tiles=tiles.format(**map_id),
        attr="Google Earth Engine",
        name="Cluster {}".format(label),
        overlay=True,
        control=True
    )
    map.add_child(folium_layer)

# Display the map
map.add_child(folium.LayerControl())
map

 

Cluster 0 mean NDVI: 0.053956044627419764
Cluster 1 mean NDVI: 0.30094223848171353
Cluster 2 mean NDVI: 0.14687760304467612


In [28]:
import folium
import numpy as np

lat = 33.653263
lon = 73.381615
zoom = 10
map = folium.Map(location=[lat, lon], zoom_start=zoom)

# Define the style for each cluster layer
colors = ['red', 'blue', 'green']

for i, color in enumerate(colors):
    # Create a random cluster layer and set its color based on the current iteration
    layer = folium.features.Circle(
        location=[lat + np.random.randn() * 0.1, lon + np.random.randn() * 0.1],
        color=color,
        radius=500
    )
    map.add_child(layer)

map.add_child(folium.LayerControl())
map
